<a href="https://colab.research.google.com/github/AmBha21/sparta-lab/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
from google.colab import drive
drive.mount('/content/drive')

import re
from transformers import BertTokenizer, BertForMaskedLM
import torch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# load BERT large cased model from HuggingFace
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Setting left wing text variable

In [3]:
left_wing_path = '/content/drive/My Drive/sparta lab/transcripts/left_videos.txt'

with open(left_wing_path, "r") as fp:
    lines = fp.readlines()  # Read all lines into a list

portion_length = len(lines) // 100  # Calculate 1/100th of the total number of lines
partial_text = lines[:portion_length]  # Extract the first 1/8th portion

# Split the partial text into individual lines/sentences
text = [line.strip() for line in partial_text if line.strip()]

In [4]:
text[:10]

['he',
 'checked off his equipment',
 'hello my name is Abigail',
 'over the last few years I have emailed',
 "Britain's National healthcare provider",
 "133 times trying to get a doctor's",
 'appointment',
 'this video is about what that was like',
 'why big institutions fall apart and why',
 "they're so difficult to change"]

In [5]:
inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [6]:
#inputs

In [7]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [8]:
#inputs.keys()

In [9]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [10]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False,  True, False,  ..., False, False, False]])

In [11]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [12]:
selection[:5]

[[], [], [], [4, 7], []]

In [13]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [14]:
#inputs.input_ids

In [15]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [16]:
dataset = MeditationsDataset(inputs)

In [17]:
loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

In [18]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [19]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
input("Press Enter to continue...")

Press Enter to continue...


''

In [21]:
from tqdm import tqdm  # for our progress bar

epochs = 1

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/1042 [00:00<?, ?it/s]<ipython-input-15-6dccb434beb5>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 0: 100%|██████████| 1042/1042 [16:43<00:00,  1.04it/s, loss=0.0116]


In [28]:
# Example test sentences with [MASK]
test_sentences = [
    "Economic inequality is a [MASK] issue that needs immediate attention.",
    "Universal healthcare is a fundamental [MASK] of every society.",
    "Climate change is the biggest [MASK] facing our generation.",
    "The government should provide [MASK] education for everyone.",
    "Workers' rights must be [MASK] and protected.",
    "Social justice is essential for [MASK] progress.",
    "Renewable energy sources are key to achieving [MASK] sustainability.",
    "Public transport should be [MASK] and accessible to reduce carbon emissions.",
    "Affordable housing is a [MASK] right that should be available to all.",
    "Diplomacy and peace talks are preferable to [MASK] intervention in foreign conflicts."
]

# Tokenize and predict
# Assuming device is defined as 'cuda' or 'cpu'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

for sentence in test_sentences:
    inputs = tokenizer.encode(sentence, return_tensors="pt")
    inputs = inputs.to(device)  # Move inputs to the same device as the model
    mask_token_index = torch.where(inputs == tokenizer.mask_token_id)[1]

    token_logits = model(inputs).logits
    mask_token_logits = token_logits[0, mask_token_index, :]

    top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

    print(f"Original Sentence: {sentence}")
    for token in top_5_tokens:
        word = tokenizer.decode([token])
        new_sentence = sentence.replace(tokenizer.mask_token, word)
        print(f"Predicted Sentence: {new_sentence}")
    print("\n")



Original Sentence: Economic inequality is a [MASK] issue that needs immediate attention.
Predicted Sentence: Economic inequality is a complex issue that needs immediate attention.
Predicted Sentence: Economic inequality is a major issue that needs immediate attention.
Predicted Sentence: Economic inequality is a single issue that needs immediate attention.
Predicted Sentence: Economic inequality is a separate issue that needs immediate attention.
Predicted Sentence: Economic inequality is a serious issue that needs immediate attention.


Original Sentence: Universal healthcare is a fundamental [MASK] of every society.
Predicted Sentence: Universal healthcare is a fundamental part of every society.
Predicted Sentence: Universal healthcare is a fundamental thing of every society.
Predicted Sentence: Universal healthcare is a fundamental requirement of every society.
Predicted Sentence: Universal healthcare is a fundamental document of every society.
Predicted Sentence: Universal healthca